## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os
import numpy as np
# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Kuching,MY,2022-10-19 01:06:41,1.5500,110.3333,79.02,94,20,0.00,few clouds
1,1,Airai,TL,2022-10-19 01:06:41,-8.9266,125.4092,73.99,60,100,0.87,overcast clouds
2,2,Hithadhoo,MV,2022-10-19 01:06:41,-0.6000,73.0833,81.07,71,100,8.43,overcast clouds
3,3,Felanitx,ES,2022-10-19 01:05:22,39.4696,3.1483,68.13,90,100,0.65,overcast clouds
4,4,Salalah,OM,2022-10-19 01:06:42,17.0151,54.0924,71.69,53,0,2.30,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 95


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# 4a. Determine if there are any empty rows.

preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"]>=min_temp) & (city_data_df["Max Temp"]<=max_temp)]
preferred_cities_df.isnull().sum()


City_ID                0
City                   0
Country                2
Date                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [5]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.isnull().sum()

City_ID                0
City                   0
Country                0
Date                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Kuching,MY,79.02,few clouds,1.5500,110.3333,
1,Airai,TL,73.99,overcast clouds,-8.9266,125.4092,
2,Hithadhoo,MV,81.07,overcast clouds,-0.6000,73.0833,
4,Salalah,OM,71.69,clear sky,17.0151,54.0924,
9,Butaritari,KI,81.41,overcast clouds,3.0707,172.7902,
13,Tiarei,PF,82.00,light rain,-17.5333,-149.3333,
14,Mahebourg,MU,70.09,scattered clouds,-20.4081,57.7000,
15,Hervey Bay,AU,71.02,moderate rain,-25.2986,152.8535,
16,Pemangkat,ID,79.63,overcast clouds,1.1667,108.9667,
19,Padang,ID,80.65,overcast clouds,-0.9492,100.3543,


In [7]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found...skipping the city.")
        hotel_df.loc[index, "Hotel Name"] = np.nan

Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.


In [8]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(axis="index", how="any")
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Kuching,MY,79.02,few clouds,1.5500,110.3333,Hilton Kuching
2,Hithadhoo,MV,81.07,overcast clouds,-0.6000,73.0833,Scoop Guest House
4,Salalah,OM,71.69,clear sky,17.0151,54.0924,"HAMDAN PLAZA HOTEL SALALAH, an HTG Hotel"
9,Butaritari,KI,81.41,overcast clouds,3.0707,172.7902,Isles Sunset Lodge
13,Tiarei,PF,82.00,light rain,-17.5333,-149.3333,Le Rocher de Tahiti
14,Mahebourg,MU,70.09,scattered clouds,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
15,Hervey Bay,AU,71.02,moderate rain,-25.2986,152.8535,Shelly Bay Resort
16,Pemangkat,ID,79.63,overcast clouds,1.1667,108.9667,Hotel Duyung Permai 2
19,Padang,ID,80.65,overcast clouds,-0.9492,100.3543,Mercure Padang
24,Hambantota,LK,77.34,overcast clouds,6.1241,81.1185,Bungalow 63


In [9]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")


In [10]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [11]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))